# Objectiv in 2 minutes

In [ ]:
from bach_open_taxonomy.modelhub.modelhub import ModelHub

## Model-ready data
- No tracking plan
- No cleaning
- No transformation

In [ ]:
modelhub = ModelHub(time_aggregation='YYYY-MM-DD', start_date='2021-11-16')
df = modelhub.from_objectiv_data()
df[df.event_type == 'PressEvent'].head()

## Talk Pandas, run SQL on full dataset
- SQL on your full database 
- Popular pandas-like operations

In [ ]:
example = df.groupby(['user_id']).agg({'event_type':'nunique','session_hit_number':'count'})
example.head()

In [ ]:
print(example.view_sql())

## Taxonomy created for modeling
- See the UI location of an event
- Strict typing of events

```python
of.location_stack
```
```json
[{"id": "docs", "_type": "RootLocationContext"},
 {"id": "docs-sidebar", "_type": "NavigationContext"},
 {"id": "API Reference", "_type": "ExpandableContext"},
 {"id": "DataFrame", "_type": "ExpandableContext"},
 {"id": "Overview", "_type": "LinkContext"}]
```

## Pre-built models 
- For common product analytics use-cases
- Chain them together

In [ ]:
modelhub.add_conversion_event(location_stack=df.location_stack.json[{'id': 'Quickstart Guide', '_type': 'LinkContext'}:],
                              event_type='PressEvent',
                              name='quickstart_presses')
conversion_events = modelhub.map.is_conversion_event(df, 'quickstart_presses')

In [ ]:
df['is_new_user_month'] = modelhub.map.is_new_user(df, time_aggregation = 'YYYY-MM')
df['is_twice_converted'] = modelhub.map.conversion_count(df, name='quickstart_presses')==2
modelhub.aggregate.unique_users(df[df.is_new_user_month & df.is_twice_converted]).head()

In [ ]:
print(modelhub.aggregate.session_duration(df).view_sql())

## To Pandas
- When results are smaller, move to pandas
- Add any pandas supported ML

In [ ]:
example.to_pandas().head()

## To Metabase
- Share dashboards with your team
- Creates SQL and settings for Metabase

In [ ]:
df['is_new_user'] = modelhub.map.is_new_user(df)
modelhub.to_metabase(modelhub.agg.unique_users(df[df.is_new_user]), 
                     config={'name': 'Unique New Users'})